In [1]:
import bz2 

train_file = bz2.BZ2File('/kaggle/input/amazonreviews/train.ft.txt.bz2')
test_file = bz2.BZ2File('/kaggle/input/amazonreviews/test.ft.txt.bz2')

train_file = train_file.readlines()
test_file = test_file.readlines()

In [2]:
from tqdm import tqdm 

num_train = 800000  # We're training on the first 800,000 reviews in the dataset
num_test = 200000  # Using 200,000 reviews from test set

train_file = [x.decode('utf-8') for x in tqdm(train_file[:num_train])]
test_file = [x.decode('utf-8') for x in tqdm(test_file[:num_test])]


100%|██████████| 200000/200000 [00:00<00:00, 815576.10it/s]


In [3]:
print(train_file[0])
print(test_file[0])

__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^

__label__2 Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"



In [4]:
train_labels = [0 if x.split()[0] == "__label__1" else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file] # Get the sequences without upper case and '\n'

test_labels = [0 if x.split()[0] == "__label__1" else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]

In [5]:
import re 

# convert all intergers to zero 
for i in tqdm(range(len(train_sentences))):
    train_sentences[i] = re.sub('\d', '0', train_sentences[i])

for i in tqdm(range(len(test_sentences))):
    test_sentences[i] = re.sub('\d', '0', test_sentences[i])
    
for i in tqdm(range(len(train_sentences))):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com ' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
for i in tqdm(range(len(test_sentences))):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com ' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

100%|██████████| 200000/200000 [00:00<00:00, 321553.77it/s]


In [6]:
print(train_sentences[0])
print(test_sentences[0])

stuning even for the non-gamer: this sound track was beautiful! it paints the senery in your mind so well i would recomend it even to people who hate vid. game music! i have played the game chrono cross but out of all of the games i have ever played it has the best music! it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen! ^_^
great cd: my lovely pat has one of the great voices of her generation. i have listened to this cd for years and i still love it. when i'm in a good mood it makes me feel better. a bad mood just evaporates like sugar in the rain. this cd just oozes life. vocals are jusat stuunning and lyrics just kill. one of life's hidden gems. this is a desert isle cd in my book. why she never made it big is just beyond me. everytime i play this, no matter black, white, young, old, male, female everybody says one thing "who was that singing ?"


In [7]:
from collections import Counter
import nltk 

words = Counter()
for i, sentence in enumerate(train_sentences):
    # The sentences will be stored as a list of words/tokens
    train_sentences[i] = []
    for word in nltk.word_tokenize(sentence):
        words.update([word])
        train_sentences[i].append(word)
    if i % 40000 == 0:
        print("Finished : %5.2f %%" %(i*100/num_train))
print("Done!")

Finished :  0.00 %
Finished :  5.00 %
Finished : 10.00 %
Finished : 15.00 %
Finished : 20.00 %
Finished : 25.00 %
Finished : 30.00 %
Finished : 35.00 %
Finished : 40.00 %
Finished : 45.00 %
Finished : 50.00 %
Finished : 55.00 %
Finished : 60.00 %
Finished : 65.00 %
Finished : 70.00 %
Finished : 75.00 %
Finished : 80.00 %
Finished : 85.00 %
Finished : 90.00 %
Finished : 95.00 %
Done!


In [8]:
# Remove the words that only appear once 
words = {k : v for k,v in words.items() if v>1}
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD', '_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {w : i for i,w in enumerate(words)}
idx2word = {i : w for i,w in enumerate(words)}

In [9]:
print(len(word2idx))

226162


In [10]:
for i, sentence in enumerate(train_sentences):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_sentences):
    # For test sentences, we have to tokenize the sentences as well
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]

In [11]:
import numpy as np 

def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len), dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

seq_len = 200 

train_sentences = pad_input(train_sentences, seq_len)
test_sentences = pad_input(test_sentences, seq_len)

# Converting our labels into numpy arrays 
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [12]:
split_frac = 0.2 # 20% validation 
split_id = int(split_frac*len(train_labels))
val_sentences, train_sentences = train_sentences[:split_id], train_sentences[split_id:]
val_labels , train_labels = train_labels[:split_id], train_labels[split_id:]

In [13]:
import torch 
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn 

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

batch_size = 400 

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden


In [16]:
vocab_size = len(word2idx) + 1 
output_size = 1 
embedding_dim = 400
hidden_dim = 512
n_layers = 2 

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers).to(device)
lr = 0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [17]:
epochs = 2 
counter = 0 
print_every = 500
clip = 5
val_loss_min = np.Inf

model.train()
for epoch in range(epochs):
    h = model.init_hidden(batch_size)
    for inputs, labels in train_loader:
        counter += 1
        model.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        h = tuple([e.data for e in h])
        out, h = model(inputs, h)
        loss = criterion(out.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter % print_every == 0:
            model.eval()
            val_losses = []
            val_h = model.init_hidden(batch_size)
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                val_h = tuple([e.data for e in val_h])
                out, val_h = model(inputs, val_h)
                val_loss = criterion(out.squeeze(), labels.float())
                val_losses.append(val_loss.item())

            print("\nEPOCH %d/%d... \t Step: %d... \t Training Loss: %.4f... \t Validation Loss: %.4f..." %(epoch+1, epochs,
                                                                                      counter, loss.item(), np.mean(val_losses)
                                                                                     ))
            if np.mean(val_losses) < val_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_min,np.mean(val_losses)))
                torch.save(model.state_dict(), 'model.pt')
                val_loss_min = np.mean(val_losses)

        model.train()
        


EPOCH 1/2... 	 Step: 500... 	 Training Loss: 0.1965... 	 Validation Loss: 0.2072...
Validation loss decreased (inf --> 0.207180).  Saving model ...

EPOCH 1/2... 	 Step: 1000... 	 Training Loss: 0.1319... 	 Validation Loss: 0.1864...
Validation loss decreased (0.207180 --> 0.186375).  Saving model ...

EPOCH 1/2... 	 Step: 1500... 	 Training Loss: 0.1976... 	 Validation Loss: 0.1776...
Validation loss decreased (0.186375 --> 0.177598).  Saving model ...

EPOCH 2/2... 	 Step: 2000... 	 Training Loss: 0.1417... 	 Validation Loss: 0.1854...

EPOCH 2/2... 	 Step: 2500... 	 Training Loss: 0.1959... 	 Validation Loss: 0.1774...
Validation loss decreased (0.177598 --> 0.177352).  Saving model ...

EPOCH 2/2... 	 Step: 3000... 	 Training Loss: 0.1749... 	 Validation Loss: 0.1778...


In [18]:
# Loading the best model
model.load_state_dict(torch.load('/kaggle/working/model.pt'))

test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Test loss: 0.179
Test accuracy: 93.141%


In [ ]:
print("ok")